In [1]:
import ctypes
import numpy as np
from numba import cuda

In [3]:
# a=np.arange(6).reshape(2,3).astype(np.float32)
a = np.random.randn(2,3).astype(np.float32)
# ad=a.ravel('F')
af=np.asfortranarray(a)

In [4]:
# b=np.arange(12).reshape(3,4).astype(np.float32)
b = np.random.randn(3,4).astype(np.float32)
# bd=b.ravel('F')
bf=np.asfortranarray(b)

In [5]:
c=np.empty((a.shape[0],b.shape[1]),dtype=np.float32)
# cd=c.ravel('F')
cf=np.asfortranarray(c)

In [6]:
af.shape,bf.shape,cf.shape

((2, 3), (3, 4), (2, 4))

In [7]:
a.shape,b.shape,c.shape

((2, 3), (3, 4), (2, 4))

In [8]:
sgemm=ctypes.CDLL('libsgemm.so')
al,bet,b0=1,0,np.arange(1,dtype=np.float32)

In [9]:
m=a.shape[0]
n=b.shape[1]
k=b.shape[0]
lda=a.shape[0]
ldb=b.shape[0]
ldc=c.shape[0]
print(m,n,k)
print(lda,ldb,ldc)

200 400 300
200 300 200


In [115]:
%%time
da=cuda.to_device(a).T
db=cuda.to_device(b).T
dc=cuda.to_device(c).T

CPU times: user 551 ms, sys: 3.09 ms, total: 554 ms
Wall time: 555 ms


In [116]:
%%time
sgemm.createHandle()
sgemm.gemm(ord('N'),ord('N'),
           da.device_ctypes_pointer,
           db.device_ctypes_pointer,
           dc.device_ctypes_pointer,
           ctypes.c_int(m),ctypes.c_int(n),ctypes.c_int(k),
           ctypes.c_int(lda),ctypes.c_int(ldb),ctypes.c_int(ldc),
           ctypes.c_float(al),ctypes.c_float(bet),
           ctypes.c_void_p(b0.ctypes.data))
sgemm.destroyHandle()

CPU times: user 3.07 ms, sys: 0 ns, total: 3.07 ms
Wall time: 1.79 ms


0

In [117]:
# %%time
# sgemm.gemm(ord('N'),ord('N'),ctypes.c_void_p(ad.ctypes.data),ctypes.c_void_p(bd.ctypes.data),ctypes.c_void_p(cd.ctypes.data),ctypes.c_int(m),ctypes.c_int(n),ctypes.c_int(k),ctypes.c_int(lda),ctypes.c_int(ldb),ctypes.c_int(ldc),ctypes.c_float(al),ctypes.c_float(bet),ctypes.c_void_p(b0.ctypes.data))

In [118]:
%%time
d=a.dot(b)

CPU times: user 24.8 ms, sys: 0 ns, total: 24.8 ms
Wall time: 8.59 ms


In [119]:
d.ravel(order='F').astype(int)

array([  0, -15,  35, ...,  -4, -31, -21])

In [120]:
da.shape,db.shape,dc.shape

((300, 200), (400, 300), (400, 200))

In [121]:
cd=dc.T.copy_to_host()

In [124]:
abs(d-cd).sum(),abs(d-cd).mean(),abs(d-cd).max()

(0.0, 0.0, 0.0)

In [122]:
cd

array([[ -0.3370665, -29.300005 ,  24.00332  , ...,  21.63932  ,
          8.458516 ,  22.585714 ],
       [-15.905303 ,   4.8357763,   1.8553208, ...,   8.749517 ,
          5.1911225,  19.564985 ],
       [ 35.001812 ,   1.5046694,  -6.926655 , ..., -54.81104  ,
          8.110646 ,  40.04506  ],
       ...,
       [ -9.220782 ,  11.580519 ,  -9.026348 , ...,   0.715599 ,
        -18.861017 ,  -4.322012 ],
       [-11.318473 ,  -6.420425 ,   5.778888 , ...,  24.769531 ,
         18.909126 , -31.591375 ],
       [  8.513818 ,   1.2622813,   8.707471 , ...,   3.2444088,
         -5.0013027, -21.488716 ]], dtype=float32)

In [123]:
d

array([[ -0.3370665, -29.300005 ,  24.00332  , ...,  21.63932  ,
          8.458516 ,  22.585714 ],
       [-15.905303 ,   4.8357763,   1.8553208, ...,   8.749517 ,
          5.1911225,  19.564985 ],
       [ 35.001812 ,   1.5046694,  -6.926655 , ..., -54.81104  ,
          8.110646 ,  40.04506  ],
       ...,
       [ -9.220782 ,  11.580519 ,  -9.026348 , ...,   0.715599 ,
        -18.861017 ,  -4.322012 ],
       [-11.318473 ,  -6.420425 ,   5.778888 , ...,  24.769531 ,
         18.909126 , -31.591375 ],
       [  8.513818 ,   1.2622813,   8.707471 , ...,   3.2444088,
         -5.0013027, -21.488716 ]], dtype=float32)

In [90]:
np.allclose(d,cd)

True

In [2]:
gg=np.empty((11900,20000),dtype=np.float32)
gg[:]=np.random.rand(*gg.shape)
gg.size*4/1024/1024

907.89794921875

In [3]:
%%time
dgg=cuda.to_device(gg)#.T.reshape(gg.shape)

CPU times: user 428 ms, sys: 194 ms, total: 622 ms
Wall time: 662 ms


In [4]:
%%time
dgt=cuda.device_array_like(gg)

CPU times: user 0 ns, sys: 2.36 ms, total: 2.36 ms
Wall time: 1.74 ms


In [5]:
sgemm=ctypes.CDLL('libsgemm.so')
al,bet=1,0
m,n=gg.shape

In [6]:
%%time
sgemm.createHandle()
sgemm.transpose(dgg.device_ctypes_pointer,
                dgt.device_ctypes_pointer,
                ctypes.c_int(n),ctypes.c_int(m),
                ctypes.c_float(al),ctypes.c_float(bet))
sgemm.destroyHandle()

CPU times: user 294 ms, sys: 108 ms, total: 402 ms
Wall time: 534 ms


In [7]:
(dgt.copy_to_host().ravel('K')==gg.ravel('F')).all()

True

In [12]:
del dgg
del dgt